In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report

In [28]:
df = pd.read_csv('../../ML_data/dataset/internet_service_churn.csv')
df.head()

,id,is_tv_subscriber,is_movie_package_subscriber,subscription_age,bill_avg,reamining_contract,service_failure_count,download_avg,upload_avg,download_over_limit,churn
0,15,1,0,11.95,25,0.14,0,8.4,2.3,0,0
1,18,0,0,8.22,0,NaN,0,0.0,0.0,0,1
2,23,1,0,8.91,16,0.00,0,13.7,0.9,0,1
3,27,0,0,6.87,21,NaN,1,0.0,0.0,0,1
4,34,0,0,6.39,0,NaN,0,0.0,0.0,0,1


In [29]:
# reamining_contract 결측치 0으로 대체
# subscription_age = -0.02 제거
# download_avg, upload_avg 결측치 제거

In [30]:
df.isna().sum()

id                                 0
is_tv_subscriber                   0
is_movie_package_subscriber        0
subscription_age                   0
bill_avg                           0
reamining_contract             21572
service_failure_count              0
download_avg                     381
upload_avg                       381
download_over_limit                0
churn                              0
dtype: int64

In [31]:
df['reamining_contract'] = df['reamining_contract'].fillna(0)

In [32]:
df = df[df['subscription_age'] >= 0]

In [33]:
df = df.dropna(subset=['download_avg'])

In [34]:
df.isna().sum()

id                             0
is_tv_subscriber               0
is_movie_package_subscriber    0
subscription_age               0
bill_avg                       0
reamining_contract             0
service_failure_count          0
download_avg                   0
upload_avg                     0
download_over_limit            0
churn                          0
dtype: int64

# Baseline 모델 비교
- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier

In [ ]:
# !pip install catboost
# !pip install xgboost
# !pip install lightgbm

In [36]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

X = df.drop(columns=['churn'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링 (로지스틱 회귀용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(n_estimators=100, learning_rate=0.05, random_state=42),
    "CatBoost": CatBoostClassifier(iterations=100, learning_rate=0.05, depth=6,
                                   loss_function='Logloss', verbose=0, random_seed=42)
}

for name, model in models.items():
    if name == "Logistic Regression":
        model.fit(X_train_scaled, y_train)
        preds = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

    print(f"=== {name} ===")
    print("Accuracy:", round(accuracy_score(y_test, preds), 4))
    print("F1-score:", round(f1_score(y_test, preds), 4))
    print()

=== Logistic Regression ===
Accuracy: 0.9
F1-score: 0.9087

=== Decision Tree ===
Accuracy: 0.9726
F1-score: 0.9754

=== Random Forest ===
Accuracy: 0.9699
F1-score: 0.9729

=== XGBoost ===
Accuracy: 0.9834
F1-score: 0.985

[LightGBM] [Info] Number of positive: 32039, number of negative: 25474
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000829 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1405
[LightGBM] [Info] Number of data points in the train set: 57513, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557074 -> initscore=0.229296
[LightGBM] [Info] Start training from score 0.229296


/opt/miniconda3/envs/tf311/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [12:15:43] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


=== LightGBM ===
Accuracy: 0.9716
F1-score: 0.9745

=== CatBoost ===
Accuracy: 0.948
F1-score: 0.9532



In [37]:
# 교차검증
from sklearn.model_selection import cross_val_score

# 모델 목록 정의
models = {
    "LightGBM": LGBMClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42)
}

# 5-Fold 교차검증 수행 (F1-score 기준)
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='f1')
    print(f"=== {name} ===")
    print(f"Fold별 F1-score: {scores}")
    print(f"평균 F1-score: {scores.mean():.4f}")
    print()

[LightGBM] [Info] Number of positive: 32039, number of negative: 25474
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000747 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1407
[LightGBM] [Info] Number of data points in the train set: 57513, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.557074 -> initscore=0.229296
[LightGBM] [Info] Start training from score 0.229296
[LightGBM] [Info] Number of positive: 32039, number of negative: 25474
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000751 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1407
[LightGBM] [Info] Number of data points in the train set: 57513, number of used features: 10
[LightGBM] [Info] [b

/opt/miniconda3/envs/tf311/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [12:15:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/miniconda3/envs/tf311/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [12:15:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/miniconda3/envs/tf311/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [12:15:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/opt/miniconda3/envs/tf311/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [12:15:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" }

=== XGBoost ===
Fold별 F1-score: [0.71607366 0.6934678  0.65405457 0.57477414 0.02075612]
평균 F1-score: 0.5318

=== RandomForest ===
Fold별 F1-score: [7.15753731e-01 6.91271183e-01 6.69875503e-01 5.70219124e-01
 2.49656722e-04]
평균 F1-score: 0.5295



F1-score가 0.97에서 0.53까지 떨어짐 -> Overfitting !!

 📊 모델 성능 요약

- **Logistic Regression**: 정확도 0.90, F1-score 0.91로 비교적 안정적인 성능을 보임  
- **Decision Tree**: 정확도 0.97, F1-score 0.98로 매우 높은 수치를 기록  
- **Random Forest**: 정확도 0.97, F1-score 0.97로 Decision Tree와 유사한 성능을 보임  
- **XGBoost**: 정확도 0.98, F1-score 0.99로 가장 우수한 결과를 나타냄  
- **LightGBM**: 정확도 0.97, F1-score 0.97로 높은 성능을 기록  

---

🔎 교차검증 결과 분석 (5-Fold Cross Validation)

- 일반화 성능을 검증하기 위해 **Random Forest**, **XGBoost**, **LightGBM** 모델을 대상으로 교차검증을 수행함  
- 그 결과, 세 모델 모두 **평균 F1-score가 약 0.52~0.53 수준으로 급격히 하락**  

➡️ 이는 학습 데이터에 대한 **과적합(overfitting)** 이 발생했음을 의미함  
➡️ 훈련 세트에서는 높은 성능을 보였지만, **새로운 데이터에 대한 일반화 성능이 충분하지 않았음**


In [38]:
#### 머신러닝 베이스 라인 모델 선정 : Random Forest
# - Decision Tree
# - Random Forest
# - Logistic Regression

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 1. 데이터 분리 (X: feature, y: target)
df_reg = pd.read_csv('../../ML_data/dataset/reg_model_preprocessed.csv')
X = df_reg.drop('churn', axis=1)
y = df_reg['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 모델 정의
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# 3. 모델 학습
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)

# 4. 모델 성능 비교
for name, model in models.items():
    print(f"\n=== {name} ===")
    y_pred = model.predict(X_test)
    print(classification_report(y_test, y_pred))
# 하이퍼파라미터를 세부적으로 조정하지 않은 기본 설정 기준으로 비교했을 때, RandomForest의 성능이 가장 높게 나타났다.


=== Decision Tree ===
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      6327
           1       0.92      0.92      0.92      8052

    accuracy                           0.91     14379
   macro avg       0.91      0.90      0.91     14379
weighted avg       0.91      0.91      0.91     14379


=== Random Forest ===
              precision    recall  f1-score   support

           0       0.92      0.94      0.93      6327
           1       0.96      0.93      0.94      8052

    accuracy                           0.94     14379
   macro avg       0.94      0.94      0.94     14379
weighted avg       0.94      0.94      0.94     14379


=== Logistic Regression ===
              precision    recall  f1-score   support

           0       0.89      0.94      0.91      6327
           1       0.95      0.91      0.93      8052

    accuracy                           0.92     14379
   macro avg       0.92      0.92      0.92     14379

In [39]:
# 머신러닝 베이스 라인 모델 : RandomForest

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# 1. 데이터 분리 (X: feature, y: target)
X = df_reg.drop('churn', axis=1)
y = df_reg['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 모델 생성
model = RandomForestClassifier(random_state=42, n_estimators=100)

# 3. 모델 학습
model.fit(X_train, y_train)

# 4. 성능 조회
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93      6327
           1       0.96      0.93      0.94      8052

    accuracy                           0.94     14379
   macro avg       0.94      0.94      0.94     14379
weighted avg       0.94      0.94      0.94     14379

